In [ ]:
HOG_CACHE = {}
def add_to_cache (img, HOG) :
    h = hash(np.array(img).tobytes())
    HOG_CACHE[h] = HOG

def get_from_cache (img) :
    key = hash(np.array(img).tobytes())
    return HOG_CACHE[key]

def in_cache (img) :
    key = hash(np.array(img).tobytes())
    return key in HOG_CACHE


def metric_chi2 (x, y) :
    return (np.sum((x - y)**2 / (x + y)) / 2)


In [ ]:
from math import floor
from os import listdir
import time

import numpy as np
from scipy.ndimage import correlate
from scipy.spatial.distance import cdist
from scipy.stats import norm

import skimage.io as skio
import skimage.util as skutil
import skimage.color as skolor

from sklearn.cluster import KMeans
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

import matplotlib.pyplot as plt


def read_collection (path, read_gray=False) :
    ''' Read a collection of images.
    
    Arguments:
    path (string)         -- the directory path
    [read_gray] (boolean) -- argument that gets passed to skimage.io.imread
                             that determines if the images are converted
                             to grayscale after being read
                   
    Returns:
    ims (ndarray) --  a list where each element is a numpy array 
                      representing a image
    '''
    
    files = listdir(path)
    ims = []
    for fname in files :
        try :
            afname = path + '/' + fname
            im = skio.imread(afname, read_gray)
            ims.append(im)
        except Exception as e :
            print('File is not a supported image format')
    
    return ims

def normalize (histograms) :
    ''' Normalizes a histogram so that the sum of its elements equals 1.
    
    Arguments:
    histograms (ndarray) -- one or a collection of histograms
    
    Returns:
    nhist (ndarray) -- the normalized histogram(s)
    '''
    
    sums = np.sum(histograms, axis=1)
    histograms = np.rollaxis(histograms, 1)
    histograms = histograms / sums
    histograms = np.rollaxis(histograms, 1)
    
    return histograms

def gradient (img) :
    ''' Computes the gradient of an image.
    
    The gradient is a 3D matrix where the first layer stores the 
    magnitude, while the second layer stores its orientation (in degrees).
    
    Arguments:
    img (ndarray) -- the matrix for which the gradient is computed
    
    Returns (NxMx2 ndarray) -- the gradient where N and M are the height and width
                               of img
    '''
    img = skolor.rgb2gray(img)
    grad = np.zeros(img.shape+(2,))
    
    f = np.array([[0, 0, 0], [-1, 0, 1], [0, 0, 0]])
    gradx = correlate(img, f, origin=1)
    grady = correlate(img, f.transpose(), origin=1)
    
    orient = np.degrees(np.arctan2(grady, gradx))+180
    mag = np.sqrt(gradx**2 + grady**2)
    
    grad[:,:,0] = mag
    grad[:,:,1] = orient
    
    return grad
    
def impatches (img, grid_shape, patch_shape=(16, 16)) :
    ''' Divides the source image into equale spaced windows.
    
    The division is made using skimage.util.view_as_windows
    that creates a sliding window over the specified image.
    
    The windows may overlap if the grid_shape is big enough.
    
    Arguments:
    img (ndarray)         -- the matrix which will get sliced
    grid_shape (tuple)    -- the number of windows along each 
                             dimension
    [patch_shape] (tuple) -- the size of each window
    
    Returns:
    patches (axbxNxM[xC] ndarray) -- the window grid where a and b
                                     are sizes of grid_shape and 
                                     N, M and C are the height, width
                                     and number of channels of the image
                                     (if applicable)
    '''

    n_vpatch = grid_shape[0]
    n_hpatch = grid_shape[1]
    imh = img.shape[0]
    imw = img.shape[1]
    
    if img.ndim == 3 :
        patch_shape = patch_shape + (3,)
    
    patchw = patch_shape[0]
    patchh = patch_shape[1]
    interval = (floor((imh-patchh)/(n_hpatch-1)),
                floor((imw-patchw)/(n_vpatch-1)))
    if img.ndim == 3 :
        interval = interval + (3,)
    
    patches = skutil.view_as_windows(np.ascontiguousarray(img), 
                                     patch_shape, interval)
    
    if patches.shape[2] == 1 :
        patches = patches.squeeze(2)
    return patches

def describe_HOG (go_pfield) :
    ''' Given an image, it generates its HOG feature vector.
    
    The descriptor returned is a 128-dimensional vector, which
    has been obtained by concatenating 16 histograms of 8 bins each.
    
    The histograms are obtained by dividing the field into an evenly
    spaced out grid, and for each window we obtain 16 histograms by
    further dividing it into a 4x4 grid.
    
    The histograms count the orientations of the gradients.
    
    Arguments:
    go_pfield (ndarray) -- the gradient orientation matrix (this
                           has been obtained with the gradient function
                           and is the second layer of the returned array)
                           
    Returns:
    hog_descriptor (1x128 ndarray) -- a 128-dimensional vector
    '''
    pshape = go_pfield[0,0].shape
    gshape = (4,4)
    cshape = (round(pshape[0]/gshape[0]),
              round(pshape[1]/gshape[1]))
    
    
    hog_vector = []
    hog_descriptor = []
    for row in go_pfield :
        for patch in row : 
            cell_grid = impatches(patch, gshape, cshape)
            for crow in cell_grid :
                for cell in crow :
                    hist = np.histogram(cell, bins=8, range=(0, 360))[0]
                    hog_vector.append(hist)
            hog_descriptor.append(np.array(hog_vector).flatten())
            hog_vector = []
    
    hog_descriptor = np.array(hog_descriptor)
    
    return hog_descriptor

def generate_vocabulary (collection, k, iter_max=100, grid_shape=(10, 10), patch_shape=16) :
    ''' Generate a visual words vocabulary.
    
    Because describing an image takes time and vocabulary genertion
    might be necesarry more than once, we store the HOG descriptor
    of each image into a cache from which we later retrieve the
    descriptors if they've been computed before.
    
    Arguments:
    collection (ndarray)   -- collection of images from which a
                              vocabulary will be generated
    k (scalar)             -- the number of words the vocabulary
                              will contain
    [iter_max] (scalar)    -- the number of iterations at which the
                              k-means algorithm should stop
    [grid_shape] (tuple)   -- the shape of the grid into which
                              each image will be divided when describing it
    [patch_shape] (scalar) -- the size of the windows of the grid (squares)
    
    Returns:
    kmeans (KMeans) -- KMeans object which can be used to obtain the words
                       and their labels.
                       words: KMeans.cluster_centers_
                       labels: KMeans.labels_
                       The labels are relevant to the collection, each
                       label corresponding to one image.
    '''
    stime = time.clock()
    descriptors = []
    for im in collection :
        if not in_cache(im) :
            gof = gradient(im)[:,:,1]
            gopf = impatches(gof, grid_shape)
            desc = describe_HOG(gopf)
            
            add_to_cache(im, desc)
        else :
            desc = get_from_cache(im)
        
        descriptors.append(desc)
    descriptors = np.array(descriptors)
    
    kmeans = KMeans(k, max_iter=iter_max)
    descriptors = descriptors.reshape(-1, 128)
    kmeans.fit(descriptors)
    
    return kmeans


def histogram_BOVW (imgs, vwdict, grid_shape=(10, 10), normalized=False) :
    ''' Generates the Bag of Visual Words histogram that describes an image.
    
    It does that by assigning each image's HOG descriptors to a 
    cluster in the dictionarie's "word space". Afterwards, it counts
    how many times each label appears for an image.
    
    Arguments:
    imgs (ndarray)       -- a collection of images
    vwdict (KMeans)      -- dictionary generated using generate_vocabulary
    [grid_shape] (tuple) -- the size of the grid into which the images
                            will be split when describing them using HOG
                            
    Returns:
    hists (NxK ndarray) -- a collection of histograms that describe
                           each image in the provided collection.
                           N is the number of images and K is the number
                           of words.
    '''
    desc = []
    for img in imgs : 
        if not in_cache(img) :
            gof = gradient(img)[:,:,1]
            gopf = impatches(gof, grid_shape)
            dc = describe_HOG(gopf)
            add_to_cache(img, dc)
        else : 
            dc = get_from_cache(img)
        desc.append(dc)
    # we reshape it so that each line contains a descriptor
    # instead of grid_shape width * grid_shape height
    desc = np.array(desc).reshape(-1, 128)
        
    
    labels = vwdict.predict(desc)
    nwords = len(vwdict.cluster_centers_)
    # we reshape the labels so that each line contains
    # the labels for one image
    labels = labels.reshape(-1, grid_shape[0]*grid_shape[1])
    hists = [] 
    for l in labels : 
        hist = np.histogram(l, nwords, range=(0, nwords), density=normalized)[0]
        hists.append(hist)
    hists = np.array(hists)
    
    return hists


def classify_CN (collection, phcoll_BOVW, nhcoll_BOVW, metric='euclidean') :
    ''' Classifies the collection using a nearest neighbour classifier.
    
    Arguments:
    collection (ndarray)  -- collection of BOVW histograms
    phcoll_BOVW (ndarray) -- training collection that represent 
                             positive samples of the data
    nhcoll_BOVW (ndarray) -- training collection that represent
                             negative samples of the data
    
    Returns:
    labels (1xN ndarray) -- labels for each element in the collection.
                            A True label classifies the element as 
                            being similar to the ones in the positive
                            training set, while a False label 
                            classifies it as being part of the negative
                            set.
    '''
    collection = normalize(collection)
    phcoll_BOVW = normalize(phcoll_BOVW)
    nhcoll_BOVW = normalize(nhcoll_BOVW)
    
#     pdistmat = cdist(collection, phcoll_BOVW, metric)
#     ndistmat = cdist(collection, nhcoll_BOVW, metric)
    
#     pmindist_mat = np.min(pdistmat, axis=1)
#     nmindist_mat = np.min(ndistmat, axis=1)
    
#     labels = np.less(pmindist_mat, nmindist_mat)

    pncollection = np.vstack((phcoll_BOVW, nhcoll_BOVW))
    distmat = cdist(collection, pncollection, metric)
    
    distmat = np.argsort(distmat, axis=1)[:,:5]
    labels = np.sum(distmat < phcoll_BOVW.shape[0], axis=1) >= 3
    
    return labels


def natdist_parameters (collection) :
    ''' Returns the infered parameters of a natural distribution
    
    If the supplied data is a matrix, the means and deviations
    will be computed along each column.
    
    Arguments:
    collection (ndarray) -- array (or matrix) that contain
                            the data points.
    
    Returns:
    means      (scalar, ndarray) -- the mean of the data set
    deviations (scalar, ndarray) -- the deviations of the data set.
                                    If the value was a 0, it was replaced
                                    with a 1.
    '''
    means = np.mean(collection, axis=0)
    
    deviations = np.std(collection, axis=0)
    deviations[deviations==0] = 1
    
    return means,deviations

def classify_bayes (collection, phists, nhists) :
    ''' Classifies the collection using a Naive Bayes Classifier
    
    The supplied histograms shouldn't be normalized.
    
    Arguments:
    collection (ndarray)  -- collection of BOVW histograms
    phists (ndarray)      -- training collection that represent 
                             positive samples of the data
    nhists (ndarray)      -- training collection that represent
                             negative samples of the data
                             
                             
    Returns:
    labels (1xN ndarray) -- labels for each element in the collection.
                            A True label classifies the element as 
                            being similar to the ones in the positive
                            training set, while a False label 
                            classifies it as being part of the negative
                            set.
    '''
    pmu, psigma = natdist_parameters(phists)
    nmu, nsigma = natdist_parameters(nhists)
    
    p_hist_masina = np.prod(norm.pdf(collection, pmu, psigma), axis=1)
    p_hist_nonmasina = np.prod(norm.pdf(collection, nmu, nsigma), axis=1)

    p_masina_hist = p_hist_masina / (p_hist_masina + p_hist_nonmasina)
    
    return p_masina_hist > 0.5

def classify_svm (collection, phists, nhists) :
    collection = normalize(collection)
    phists = normalize(phists)
    nhists = normalize(nhists)    
    cl = SVC()
    
    tdata = np.concatenate((phists, nhists), axis=0)
    labels = np.concatenate((np.ones(phists.shape[0]), np.zeros(nhists.shape[0])))
    cl.fit(tdata, labels)
    
    return cl.predict(collection)
    

In [ ]:
pcollection = read_collection('data/masini-exempleAntrenare-pozitive/')
ncollection = read_collection('data/masini-exempleAntrenare-negative/')
pncollection = pcollection + ncollection
testpcollection = read_collection('data/masini-exempleTestare-pozitive/')
testncollection = read_collection('data/masini-exempleTestare-negative/')

print('Starting generation of dictionary')
stime = time.clock()

dictsizes = [5, 10, 25, 50, 100, 200, 500, 1000]
dictionary = []
for k in dictsizes :
    print('Generating vocabulary (k={})...'.format(k))
    v = generate_vocabulary(pncollection, k)
    dictionary.append(v)
    print('Done')
print('Done generating the dictionary: {}ms'.format(time.clock() - stime))
print('='*20)

In [ ]:
def efficacy (labels, target) :
    return np.sum(labels == target) / labels.size

def test_method (method, name) :
    print(name + '...')
    stime = time.clock()
    plabels = method(tphists, phists, nhists)
    t = time.clock() - stime
    nlabels = method(tnhists, phists, nhists)
    e = efficacy(plabels, 1)/2 + efficacy(nlabels, 0)/2
    print('Done: {}ms (50 data points), {} efficacy'.format(t, e))
    
    return t, e

stats = {
    'nns' : {
        'times': [],
        'efficacies': []
    },
    'bayes' : {
        'times': [],
        'efficacies': []
    },
    'svm' : {
        'times': [],
        'efficacies': []
    }
}
print('Starting profiling')
for vocab in dictionary : 
    print('='*20,'='*20, sep='\n')
    print('Vocabulary size: {}'.format(len(vocab.cluster_centers_)))
    print('='*20,'='*20, sep='\n')
    print('Starting description of sample data...')
    stime = time.clock()
    phists = histogram_BOVW(pcollection, vocab)
    nhists = histogram_BOVW(ncollection, vocab)
    pnhists = histogram_BOVW(pncollection, vocab)
    tphists = histogram_BOVW(testpcollection, vocab)
    tnhists = histogram_BOVW(testncollection, vocab)
    print('Done: {}ms'.format(time.clock() - stime))
    print('='*20)
    
    t, e = test_method(classify_CN, 'Nearest Neighbour Classifier')
    stats['nns']['times'].append(t)
    stats['nns']['efficacies'].append(e)
    
    t, e = test_method(classify_bayes, 'Naive Bayes Classifier')
    stats['bayes']['times'].append(t)
    stats['bayes']['efficacies'].append(e)
    
    t, e = test_method(classify_svm, 'Support Vector Machine')
    stats['svm']['times'].append(t)
    stats['svm']['efficacies'].append(e)
    
    print('\n\n')
    
    

In [ ]:
print(np.array(stats['nns']['times']))
print(np.array(stats['nns']['efficacies']), end='\n\n\n')

print(np.array(stats['bayes']['times']))
print(np.array(stats['bayes']['efficacies']), end='\n\n\n')

print(np.array(stats['svm']['times']))
print(np.array(stats['svm']['efficacies']), end='\n\n\n')